In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------


Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 7

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v04"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v04 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V07/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V07/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M08D09 UTC 13:34


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//127443_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//145102_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//142625_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//153021_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132895_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//157807_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//162203_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//181976_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//131902_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//155562_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//122784_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111011_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115801_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//172497_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//131908_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//172525_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//122810_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//153029_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115808_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//145109_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//224002_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//181993_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//172526_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111019_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//162300_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132930_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//153030_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//127461_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//116004_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//172545_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132946_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//183000_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//122848_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//145207_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//162443_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132160_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//153039_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//127468_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//155578_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111045_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//224009_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//142642_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//155615_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//145404_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//232301_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//162450_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111070_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//153048_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//225116_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132954_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132223_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117100_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//172554_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//122857_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//211015_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//127486_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//153055_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//225150_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//122911_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132971_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//155633_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//158216_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//172570_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//132230_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//127494_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//232309_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//142658_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117108_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//162468_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//211032_V07.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742974_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//782805_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//642478_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914507_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//772403_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//713222_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//753246_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911103_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//722804_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//664024_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//652501_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742415_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//811902_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//782806_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742982_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//725998_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//782950_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//723010_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914706_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//811909_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742423_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911214_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//753280_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//782960_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//831958_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//772601_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//664043_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742983_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914807_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//726016_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742441_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//723073_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//753307_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911262_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//782998_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742992_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//832103_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914808_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//812109_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//772609_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//822516_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//726017_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//743003_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//723112_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//832422_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//753408_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911286_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//726072_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742460_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//822544_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//743004_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//812301_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//772809_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//723113_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//832423_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911287_V07.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//726084_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//723160_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//812430_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//822610_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//832446_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//751206_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742491_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//726085_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//753621_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//812440_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//822621_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//751207_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//832447_V07.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//742492_V07.csv
/volumes/data/Y2018M

In [14]:
#!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [15]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111011_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111012_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111013_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111014_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111015_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111030_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111017_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111016_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//111018_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112130_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112170_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112180_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112160_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112190_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112200_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112400_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112300_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//112500_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114443_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114444_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114445_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114446_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114447_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114448_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114449_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114450_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//114460_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115424_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115425_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115426_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115427_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115428_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115429_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115430_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115440_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//115451_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117103_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117104_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117105_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117106_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117107_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117108_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117109_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117201_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//117202_V07.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863013_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863014_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863015_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863016_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863017_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863018_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863019_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863020_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//863030_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911408_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911409_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911500_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911510_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911520_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911530_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911540_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911550_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//911560_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912400_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912510_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912520_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912530_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912540_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912551_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912552_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912553_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//912554_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913473_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913474_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913475_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913476_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913477_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913478_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913479_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913480_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//913490_V07.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914501_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914502_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914503_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914504_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914505_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914506_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914507_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914508_V07.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V07//914509_V07.csv [Content-Type=text/

In [16]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:42:14.565295


Previous runs:  
0:42:01.416597  
0:42:14.565295

